In [29]:
from autogluon import TabularPrediction as task

In [30]:
data= task.Dataset(file_path = '../Data/Indicator_based_RL/indicator_dataset.csv')

Loaded data from: ../Data/Indicator_based_RL/indicator_dataset.csv | Columns = 284 / 284 | Rows = 503452 -> 503452


In [31]:
train = data.iloc[:int(len(data) * 0.5), 1:-25]
test = data.iloc[int(len(data) * 0.5):, 1:-25]
del data

In [32]:
targets = task.Dataset(file_path = '../Data/Label_based_RL/new_labels.csv')

Loaded data from: ../Data/Label_based_RL/new_labels.csv | Columns = 21 / 21 | Rows = 503452 -> 503452


In [33]:
targets.head()

,peaks_0.5,valleys_0.5,peaks_0.25,valleys_0.25,peaks_1,valleys_1,peaks_2,valleys_2,label_0.25,reversed_label_0.25,...,reversed_label_0.5,label_1,reversed_label_1,label_2,reversed_label_2,reg_labels_0.25,reg_labels_0.5,reg_labels_1,reg_labels_2,price
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,1,0,2.13,2.13,3.53,16.56,135.38
1,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,2.24,2.24,3.64,16.67,135.27
2,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,2.02,2.02,3.42,16.45,135.49
3,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1.22,1.22,3.10,3.10,136.29
4,0,1,0,1,0,0,0,0,0,1,...,1,1,0,1,0,-0.64,-1.09,1.40,14.43,137.51


In [34]:
label_type = 'label'
coeff = '0.5'

label_column = '{}_{}'.format(label_type, coeff)
targets = targets[label_column]
train_targets = targets[:int(len(targets) * 0.5)]
test_targets = targets[int(len(targets) * 0.5):]

In [35]:
train[label_column] = train_targets
test[label_column] = test_targets

In [36]:
time_limits = 1.2 * 60 *60
predictor = task.fit(train_data=train, label=label_column, time_limits=time_limits, presets='best_quality', eval_metric='average_precision')

No output_directory specified. Models will be saved in: AutogluonModels/ag-20201016_120038/
Beginning AutoGluon training ... Time limit = 4320.0s
AutoGluon will save models to AutogluonModels/ag-20201016_120038/
AutoGluon Version:  0.0.14
Train Data Rows:    251726
Train Data Columns: 258
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    80144.39 MB
	Train Data (Original)  Memory Usage: 519.56 MB (0.6% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in t

	Time limit exceeded... Skipping KNeighborsClassifierUnif_STACKER_l1.
Fitting model: KNeighborsClassifierDist_STACKER_l1 ... Training model for up to 58.12s of the 57.95s of remaining time.
	Time limit exceeded... Skipping KNeighborsClassifierDist_STACKER_l1.
Fitting model: LightGBMClassifier_STACKER_l1 ... Training model for up to 46.54s of the 46.37s of remaining time.
	Ran out of time, early stopping on iteration 6. Best iteration is:
	[6]	train_set's binary_logloss: 0.334563	train_set's average_precision: 0.997872	valid_set's binary_logloss: 0.334548	valid_set's average_precision: 0.997393
	Ran out of time, early stopping on iteration 12. Best iteration is:
	[12]	train_set's binary_logloss: 0.188744	train_set's average_precision: 0.997978	valid_set's binary_logloss: 0.192103	valid_set's average_precision: 0.997533
	Ran out of time, early stopping on iteration 25. Best iteration is:
	[24]	train_set's binary_logloss: 0.0839523	train_set's average_precision: 0.998114	valid_set's binar

In [37]:
prediction = predictor.predict_proba(test)

In [38]:
perf = predictor.evaluate_predictions(y_true=test_targets, y_pred=prediction, auxiliary_metrics=True)

Evaluation: average_precision on test data: 0.48184363966964505
Evaluations on test data:
{
    "average_precision": 0.48184363966964505
}


In [39]:
import numpy as np
np.savetxt('predictions_{}_{}.csv'.format(label_type, coeff), prediction, delimiter = ',')

In [40]:
sum(test[label_column])/len(test[label_column])

0.4610767262817508